#### **Importing Libraries**

In [ ]:
import os
import pandas as pd
import seaborn as sns

from googleapiclient.discovery import build

#### **Setting up YouTube API.**

In [ ]:
api_service_name = "youtube"
api_version = "v3"

yt_api_key = os.environ["YT_API_KEY"]

### **I. Scraping Channel Statistics.**

#### Getting Channel ID's.

In [ ]:
channel_ids = [
    "UCX6OQ3DkcsbYNE6H8uQQuVA",
    "UC59ZRYCHev_IqjUhremZ8Tg",
    "UCvgfXK4nTYKudb0rFR6noLA",
    "UCc0YbtMkRdhcqwhu3Oad-lw",
]

#### Building YouTube API Service.

In [ ]:
youtube = build(api_service_name, api_version, developerKey=yt_api_key)  

#### Function to obtain channel statistics.

In [ ]:
def get_channel_statistics(youtube, channel_ids):
    data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics", id=",".join(channel_ids)
    )

    response = request.execute()

    for i in range(len(response["items"])):
        info = dict(
            channel_name=response["items"][i]["snippet"]["title"],
            subscribers=response["items"][i]["statistics"]["subscriberCount"],
            videos=response["items"][i]["statistics"]["videoCount"],
            views=response["items"][i]["statistics"]["viewCount"],
            # --------
            playlist_id=response["items"][i]["contentDetails"]["relatedPlaylists"]['uploads'],
        )

        data.append(info)

    return data

In [ ]:
channel_statistics = get_channel_statistics(youtube, channel_ids)

#### Create a dataframe. 

In [ ]:
channel_df = pd.DataFrame(channel_statistics)
channel_df

#### Change datatype from object to integer.

In [ ]:
channel_df.dtypes

In [ ]:
exclude_cols = ['channel_name']

cols_to_include = [col for col in channel_df.columns if col not in exclude_cols]

In [ ]:
channel_df[cols_to_include] = channel_df[cols_to_include].applymap(pd.to_numeric, errors='coerce')

In [ ]:
channel_df.dtypes